<a href="https://colab.research.google.com/github/anaborne/RavenPack-Data-Aggregation/blob/main/RavenPack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

# File paths
firms_file   = '20250707_unique_firms_trase.xlsx'
mapping_file = 'entity_rp_id_mappings.xlsx'
output_file  = 'rp_id_matched.xlsx'

# Read sheets
df_firms = pd.read_excel(firms_file, sheet_name=0)
df_map   = pd.read_excel(mapping_file, sheet_name=0)

# Normalization function
def normalize(text):
    if pd.isnull(text):
        return ''
    s = str(text).lower().strip()
    s = re.sub(r'[^\w\s]', '', s)     # remove punctuation
    s = re.sub(r'\s+', ' ', s)        # collapse whitespace
    return s

# Apply normalization
df_firms['exporter_norm'] = df_firms['exportergroup'].apply(normalize)
df_map  ['value_norm']    = df_map  ['data_value'].apply(normalize)

# Build a lookup of normalized mapping values to rp_entity_id
mapping_lookup = dict(zip(df_map['value_norm'], df_map['rp_entity_id']))

# Matching function: exact first, then substring
def match_rp_id(exporter_norm):
    # exact match
    if exporter_norm in mapping_lookup:
        return mapping_lookup[exporter_norm]
    # substring match: exporter_norm inside a longer mapping value
    for value_norm, rp_id in mapping_lookup.items():
        if exporter_norm and exporter_norm in value_norm:
            return rp_id
    return None

# Apply matching
df_firms['rp_entity_id_matched'] = df_firms['exporter_norm'].apply(match_rp_id)

# Prepare output
df_output = df_firms[['rp_entity_id_matched', 'exportergroup']].copy()
df_output.columns = ['rp_entity_id', 'company_name']

# Write to Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_output.to_excel(writer, sheet_name='Matched', index=False)

In [ ]:
!pip install dask[complete] openpyxl

import dask.dataframe as dd
import pandas as pd

# read matched file normally
match_df = pd.read_excel('rp_id_matched.xlsx', usecols=['rp_entity_id','company_name'])
match_ddf = dd.from_pandas(match_df, npartitions=1)

# have Dask read and parse the Excel (streamed under the hood) (the file is too large to read with pandas alone)
sent_ddf = dd.read_csv('RP_Sentiment_Data.csv', blocksize=25e6, dtype={'event_sentiment_score': 'object'})

# merge and rename
out = sent_ddf.merge(match_ddf, on='rp_entity_id', how='left')
out = out.rename(columns={'company_name':'exportergroup'})

# write to CSV
out.to_csv('final_rp_sentiment_data.csv', index=False)

['/content/final_rp_sentiment_data-0.csv']

In [ ]:
import dask.dataframe as dd
import pandas as pd

# 1. Read in the CSV
sent_ddf = dd.read_csv(
    'RP_Sentiment_Data.csv',
    blocksize=25e6,
    dtype={'event_sentiment_score': 'object',
           'composite_sentiment_score': 'object', # Corrected column name
           'rpa_date_utc': 'object'},
    assume_missing=True
)

# 2. Parse date & extract year
sent_ddf['rpa_date_utc'] = dd.to_datetime(sent_ddf['rpa_date_utc'], errors='coerce')
sent_ddf['year'] = sent_ddf['rpa_date_utc'].dt.year

# 3. Convert both scores to floats
sent_ddf['event_score'] = dd.to_numeric(sent_ddf['event_sentiment_score'], errors='coerce')
sent_ddf['comp_score']  = dd.to_numeric(sent_ddf['composite_sentiment_score'], errors='coerce') # Corrected column name

# 4. Drop rows missing _both_ scores
sent_ddf = sent_ddf[
    ~(sent_ddf['event_score'].isna() & sent_ddf['comp_score'].isna())
]

# 5. Build indicator cols for each metric
sent_ddf['neg_event'] = (sent_ddf['event_score'] < 0).fillna(False).astype(int)
sent_ddf['pos_event'] = (sent_ddf['event_score'] > 0).fillna(False).astype(int)
sent_ddf['neu_event'] = (sent_ddf['event_score'] == 0).fillna(False).astype(int)

sent_ddf['neg_comp'] = (sent_ddf['comp_score']  < 0).fillna(False).astype(int)
sent_ddf['pos_comp'] = (sent_ddf['comp_score']  == 0).fillna(False).astype(int) # Corrected comparison to 0 for neutral
sent_ddf['neu_comp'] = (sent_ddf['comp_score']  > 0).fillna(False).astype(int) # Corrected comparison to > 0 for positive


# 6. Group & sum all six indicators
agg = (
    sent_ddf
    .groupby(['year', 'rp_entity_id', 'entity_name'])
    .agg({
        'neg_event': 'sum',
        'pos_event': 'sum',
        'neu_event': 'sum',
        'neg_comp':  'sum',
        'pos_comp':  'sum',
        'neu_comp':  'sum'
    })
    .reset_index()
)

# 7. Rename to match template
agg = agg.rename(columns={
    'neg_event': 'N_event_sentiment_negative',
    'pos_event': 'N_event_sentiment_positive',
    'neu_event': 'N_event_sentiment_neutral',
    'neg_comp':  'N_composite_sentiment_negative',
    'pos_comp':  'N_composite_sentiment_positive',
    'neu_comp':  'N_composite_sentiment_neutral'
})


# 8. Compute & sort by firm then year
result_df = agg.compute()
result_df = result_df.sort_values(
    by=['entity_name','year'],
    ascending=[True, True]
).reset_index(drop=True)

# 9. Write out
result_df.to_excel('sentiment_counts_by_year.xlsx', index=False)

Index(['rpa_date_utc', 'rp_entity_id', 'entity_name', ' ',
       'event_sentiment_score', 'composite_sentiment_score'],
      dtype='object')


/usr/local/lib/python3.11/dist-packages/dask_expr/_expr.py:1471: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)


In [ ]:
import dask.dataframe as dd
import pandas as pd

# 1. Read the existing counts file
counts_df = pd.read_excel(
    'sentiment_counts_by_year.xlsx',
    engine='openpyxl'
)

# 2. Stream the large sentiment CSV with Dask
sent_ddf = dd.read_csv(
    'RP_Sentiment_Data.csv',
    blocksize=25e6,
    dtype={
        'event_sentiment_score': 'object',
        'composite_sentiment_score': 'object',
        'rpa_date_utc': 'object'
    },
    assume_missing=True
)

# 3. Parse dates and extract year
sent_ddf['rpa_date_utc'] = dd.to_datetime(sent_ddf['rpa_date_utc'], errors='coerce')
sent_ddf['year'] = sent_ddf['rpa_date_utc'].dt.year

# 4. Convert both sentiment columns to numeric
sent_ddf['event_score'] = dd.to_numeric(sent_ddf['event_sentiment_score'], errors='coerce')
sent_ddf['comp_score']  = dd.to_numeric(sent_ddf['composite_sentiment_score'], errors='coerce')

# 5. Compute yearly means for each entity
mean_ddf = (
    sent_ddf
    .groupby(['year', 'rp_entity_id', 'entity_name'])
    [['event_score', 'comp_score']]
    .mean()
    .reset_index()
)

# 6. Materialize the Dask result into pandas
mean_df = mean_ddf.compute()

# 7. Rename to the new average‐columns
mean_df = mean_df.rename(columns={
    'event_score': 'avg_event_sentiment',
    'comp_score':  'avg_composite_sentiment'
})

# 8. Merge the averages into the counts table
merged_df = counts_df.merge(
    mean_df,
    on=['year', 'rp_entity_id', 'entity_name'],
    how='left'
)

# 9. Save back to Excel (overwrites with the two new columns at the end)
merged_df.to_excel(
    'sentiment_counts_by_year.xlsx',
    index=False
)

/usr/local/lib/python3.11/dist-packages/dask_expr/_expr.py:1471: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)


In [ ]:
import pandas as pd

# 1. Read in enriched sentiment‐counts file
counts_df = pd.read_excel(
    'sentiment_counts_by_year.xlsx',
    engine='openpyxl'
)

# 2. Read in the rp_id to exportergroup mapping
match_df = pd.read_excel(
    'rp_id_matched.xlsx',
    usecols=['rp_entity_id', 'company_name'],
    engine='openpyxl'
)

# 3. Merge on rp_entity_id to pull in company_name
merged = counts_df.merge(
    match_df,
    on='rp_entity_id',
    how='left'
)

# 4. Rename the pulled-in column to exportergroups
merged.rename(columns={'company_name': 'exportergroups'}, inplace=True)

# 5. Save out a new file
merged.to_excel(
    'final_RP_sentiment_data.xlsx',
    index=False
)